## Reproduce DARTFISH results with a Pixel Decoding Method

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import starfish.display
from starfish import data, FieldOfView
from starfish.types import Features, Axes

from starfish import IntensityTable

from starfish.image import Filter
from starfish.spots import SpotFinder

sns.set_context('talk')
sns.set_style('ticks')

### Load image stack

Note that the data here corresopond to DARTFISHv1 2017. The group is actively working on improving the protocol.

In [ ]:
use_test_data = os.getenv("USE_TEST_DATA") is not None
exp = data.DARTFISH(use_test_data=use_test_data)

intensities = list()
for stack in exp.fov().iterate_image_type(FieldOfView.PRIMARY_IMAGES):

### visualization of matched barcodes
here, we 1. pick a rolony that was succesfully decoded to a gene. 2. pull out the average pixel trace for that rolony and 3. plot that pixel trace against the barcode of that gene

In [ ]:
# reshape the spot intensity table into a RxC barcode vector
pixel_traces = spot_intensities.stack(traces=(Axes.ROUND.value, Axes.CH.value))

# extract dataframe from spot intensity table for indexing purposes
pixel_traces_df = pixel_traces.to_features_dataframe()
pixel_traces_df['area'] = np.pi*pixel_traces_df.radius**2

# pick index of a barcode that was read and decoded from the ImageStack
ind = 4

# The test will error here on pixel_traces[ind,:] with an out of index error
# because we are using the test data.

# get the the corresponding gene this barcode was decoded to
gene = pixel_traces_df.loc[ind].target

# query the codebook for the actual barcode corresponding to this gene
real_barcode = exp.codebook[exp.codebook.target==gene].stack(traces=(Axes.ROUND.value, Axes.CH.value)).values[0]
read_out_barcode = pixel_traces[ind,:]

plt.plot(real_barcode, 'ok')
plt.stem(read_out_barcode)
sns.despine(offset=2)
plt.xticks(range(18))
plt.title(gene)
plt.xlabel('Index into R (0:5) and C(0:2)');